Please click run on the cell below.

Select your data folder (where the .tif files are stored).

Always work in your environment setup for jupyter and tensorflow.

use start.bat to initialise

In [21]:
import os
import tifffile
import PySimpleGUI as sg
import random

# Change these values as needed
TRAIN_RATIO = 0.9
IMAGE_CHANNEL = 0
MASK_CHANNEL = 1

def create_folders(source_folder, train_ratio=0.9):
    """
    Create folders for images and masks within the train and test folders.
    """
    # Define train and test folder paths
    train_folder = os.path.join(source_folder, 'train')
    test_folder = os.path.join(source_folder, 'test')

    # Create subdirectories for images and masks within train and test folders
    train_images = os.path.join(train_folder, 'images')
    train_masks = os.path.join(train_folder, 'masks')
    test_images = os.path.join(test_folder, 'images')
    test_masks = os.path.join(test_folder, 'masks')
    
    os.makedirs(train_images, exist_ok=True)
    os.makedirs(train_masks, exist_ok=True)
    os.makedirs(test_images, exist_ok=True)
    os.makedirs(test_masks, exist_ok=True)

    # Get list of files and shuffle them
    all_files = [f for f in os.listdir(source_folder) if f.lower().endswith('.tif')]
    random.shuffle(all_files)

    # Split the files based on the specified ratio
    split_index = int(len(all_files) * train_ratio)
    train_files = all_files[:split_index]
    test_files = all_files[split_index:]

    return train_images, train_masks, test_images, test_masks, train_files, test_files

def separate_channels(file_path, output_directory_images, output_directory_masks):
    """
    Separates the channels of a multi-channel TIFF file and saves each channel as a separate image file.
    
    Args:
        file_path (str): The path to the input multi-channel TIFF file.
        output_directory_images (str): The directory where image channels will be saved.
        output_directory_masks (str): The directory where mask channels will be saved.
    
    Returns:
        None
    """
    with tifffile.TiffFile(file_path) as tif:
        image = tif.asarray()

    if image.ndim == 4 and image.shape[1] == 2:
        for c in range(image.shape[1]):
            channel_image = image[:, c, :, :]
            channel_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}.tif"
            if c == IMAGE_CHANNEL:  # Assuming channel 0 is for images
                output_directory = output_directory_images
            elif c == MASK_CHANNEL:  # Assuming channel 1 is for masks
                output_directory = output_directory_masks
            else:
                print("unexpected nr. of channels")
            channel_file_path = os.path.join(output_directory, channel_file_name)
            tifffile.imwrite(channel_file_path, channel_image)
            print(f"Saved channel {c} to {channel_file_path}")
    else:
        print(f"Unexpected image dimensions for file {file_path}")

def process_file(file_path, output_directory_images, output_directory_masks):
    """
    Process a single file and save its channels to the specified directories.
    """
    if file_path.lower().endswith(".tif"):
        separate_channels(file_path, output_directory_images, output_directory_masks)

def main():
    """
    This function initializes a GUI window using PySimpleGUI, prompts the user to select an input directory 
    with TIFF files, and processes the selected files to separate the channels. It returns nothing.
    """
    sg.theme('Lightblue2')

    layout = [
        [sg.Text("Select the Input Directory with TIFF files:")],
        [sg.Input(expand_x=True), sg.FolderBrowse(key='input_folder')],
        [sg.Submit(), sg.Cancel()]
    ]

    window = sg.Window('TIFF Channel Separator', layout, keep_on_top=True, finalize=True, resizable=True)
    window.TKroot.minsize(420,100)

    while True:
        event, values = window.read()
        if event in (None, 'Cancel'):
            break
        if event == 'Submit':
            input_dir = values['input_folder']
            train_images, train_masks, test_images, test_masks, train_files, test_files = create_folders(input_dir)

            if os.path.isdir(input_dir):
                # Process train files
                for filename in train_files:
                    process_file(os.path.join(input_dir, filename), train_images, train_masks)
                
                # Process test files
                for filename in test_files:
                    process_file(os.path.join(input_dir, filename), test_images, test_masks)

                sg.popup('Processing Complete', 'The channels have been separated successfully!')
            else:
                sg.popup_error('Error', 'Please select a valid input directory.')
            break

    window.close()

if __name__ == "__main__":
    main()

